In [136]:
import pandas as pd
import bs4


In [137]:
diseases = pd.read_csv('diseases_with_desc.csv')
diseases.dropna(inplace=True)

In [138]:
subAttr = {'data-attrid': 'subtitle'}
descAttr = {'data-attrid': 'kc:/medicine/disease:description'}
secondaryDescAttr = {'data-attrid': 'kc:/medicine/disease:long description'}
rarityAttr = {'data-attrid': 'kc:/medicine/disease:location prevalence title'}

symptomAttr = {'data-attrid': 'kc:/medicine/disease:detailed symptoms'}

In [139]:
diseases_complete = pd.DataFrame()
diseases_complete['disease'] = diseases['disease']
diseases_complete['cleaned_disease'] = diseases['cleaned_disease']
diseases_complete.drop_duplicates('cleaned_disease', inplace=True)

In [140]:
def filterSubtitles(descDiv):
    soup = bs4.BeautifulSoup(descDiv, 'html.parser')
    subtitleElement = soup.find('div', subAttr)
    if subtitleElement is not None:
        return subtitleElement.text

def filterPrimaryDescriptions(descDiv):
    soup = bs4.BeautifulSoup(descDiv, 'html.parser')
    primaryDescElement = soup.find('div', descAttr)
    if primaryDescElement is not None:
        return primaryDescElement.text

def filterSecondaryDescriptions(descDiv):
    soup = bs4.BeautifulSoup(descDiv, 'html.parser')
    secondaryDescElement = soup.find('div', secondaryDescAttr)
    if secondaryDescElement is not None:
        return secondaryDescElement.text

def filterRarity(descDiv):
    soup = bs4.BeautifulSoup(descDiv, 'html.parser')
    rarityElement = soup.find('div', rarityAttr)
    if rarityElement is not None:
        return rarityElement.text

diseases_complete['subtitle'] = diseases['desc'].apply(filterSubtitles)
diseases_complete['primary_description'] = diseases['desc'].apply(filterPrimaryDescriptions)
diseases_complete['secondary_description'] = diseases['desc'].apply(filterSecondaryDescriptions)
diseases_complete['rarity'] = diseases['desc'].apply(filterRarity)
diseases_complete.reset_index(inplace=True)
diseases_complete.drop('index', axis=1, inplace=True)
